In [1]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF

def extract_labels(labels):
    # Map emotion labels to integers
    label_mapping = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
    return np.array([label_mapping[label] for label in labels])

class EmotionDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.data, self.labels = self.load_data(dataset_path)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
    @staticmethod
    def load_data(dataset_path):
        data = []
        labels = []

        for emotion_folder in os.listdir(dataset_path):
            emotion_path = os.path.join(dataset_path, emotion_folder)
            for img_name in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path)
                data.append(img)
                labels.append(emotion_folder)

        return np.array(data), extract_labels(labels)
    
    


In [2]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
train_path = 'FER2013/train'
absolute_train_path = os.path.abspath(train_path)

test_path = 'FER2013/test'
absolute_test_path = os.path.abspath(test_path)

train_dataset = EmotionDataset(absolute_train_path, transform=transform)
test_dataset = EmotionDataset(absolute_test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [4]:
# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda") # Use the GPU for computation
    print("Using GPU for computation!")
else:
    device = torch.device("cpu") # Use the CPU for computation
    print("Using CPU for computation!")

Using GPU for computation!


In [5]:
from torchvision import transforms, models
from facenet_pytorch import InceptionResnetV1 
import torch.nn as nn

face_model = models.face_resnet50(pretrained=False)
face_model.load_state_dict(torch.load('20180402-114759-step1196000.pth'))
face_model.eval() # Set the model to evaluation mode

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False
    
model.last_linear.weight.requires_grad = True
model.last_linear.bias.requires_grad = True

AttributeError: module 'torchvision.models' has no attribute 'face_resnet50'

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):

    model.train()
    
    train_accuracy = 0
    num_correct = 0
    num_samples = 0
    
    for inputs, labels in train_loader: 
     
        optimizer.zero_grad()
        
        # Forward pass
        inputs = inputs.to(device) 
        labels = labels.long().to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
        acc = float(num_correct) / num_samples
        train_accuracy += acc
        
    # Print epoch loss and accuracy
    avg_acc = train_accuracy / len(train_loader)            
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {avg_acc:.4f}')